# Block 5 Exercise 2: Predicting the fare of taxi rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

In [3]:
# a quick look at the data
X.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,...,pickup_borough_manhattan,pickup_borough_others,pickup_borough_queens,pickup_borough_staten_island,dropoff_borough_bronx,dropoff_borough_brooklyn,dropoff_borough_manhattan,dropoff_borough_others,dropoff_borough_queens,dropoff_borough_staten_island
0,-73.844311,40.721319,-73.841610,40.712278,1,15,17,1,6,2009,...,0,0,1,0,0,0,0,0,1,0
1,-74.016048,40.711303,-73.979268,40.782004,1,5,16,2,1,2010,...,1,0,0,0,0,0,1,0,0,0
2,-73.982738,40.761270,-73.991242,40.750562,2,18,0,4,8,2011,...,1,0,0,0,0,0,1,0,0,0
3,-73.987130,40.733143,-73.991567,40.758092,1,21,4,6,4,2012,...,1,0,0,0,0,0,1,0,0,0
4,-73.968095,40.768008,-73.956655,40.783762,1,9,7,2,3,2010,...,1,0,0,0,0,0,1,0,0,0


In [4]:
y.head()

0     4.5
1    16.9
2     5.7
3     7.7
4     5.3
Name: fare_amount, dtype: float64

### E 2.1 Split data into train and test data
* use data from 2009-1014 for training and 2015 for evaluation


In [5]:
X_train = X[X["pickup_year"]<2015]
Y_train = X[X["pickup_year"]<2015]
x_test = X[X["pickup_year"]==2015]
y_test = X[X["pickup_year"]==2015]

#test ob erfolgreich
print(X_train["pickup_year"].max())
print(X_train["pickup_year"].min())

print("-------------\n-------------")
print(x_test["pickup_year"].max())
print(x_test["pickup_year"].min())

2014
2009
-------------
-------------
2015
2015


### E 2.2 Train a Random Forest Regression Model
* train model using [RF regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* optimize the model parameters (#trees, depth, vars per split, ...) and evaluate the [MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) on train and test data
* Analyze the model error: are there typical settings where the model fails?

In [16]:
from sklearn.ensemble import RandomForestRegressor
#Estimators= Anzahl an Bäumen
regForr = RandomForestRegressor(n_estimators=20, random_state=0)
regForr.fit(X_train, Y_train)
predForr = regForr.predict(x_test)

In [7]:
print(predForr)

[[-73.9904807   40.7357037  -73.96472025 ...   0.           0.
    0.        ]
 [-73.9801052   40.73594285 -73.93572655 ...   0.           0.15
    0.        ]
 [-73.9761103   40.7529243  -73.9773238  ...   0.           0.
    0.        ]
 ...
 [-73.9880439   40.74618765 -73.9939098  ...   0.           0.
    0.        ]
 [-73.98716655  40.74368435 -73.99005765 ...   0.           0.
    0.        ]
 [-73.9800491   40.75475305 -73.97998925 ...   0.           0.
    0.        ]]


In [8]:
#mit Standard Baumzahl (50)
regForr_Default = RandomForestRegressor(n_estimators=50, random_state=0)
regForr_Default.fit(X_train, Y_train)
predForr_Default = regForr_Default.predict(x_test)

In [9]:
print(predForr_Default)

[[-73.99386884  40.7333652  -73.96626862 ...   0.           0.
    0.        ]
 [-73.97869854  40.73466254 -73.92857586 ...   0.           0.14
    0.        ]
 [-73.9793091   40.7526846  -73.98081882 ...   0.           0.
    0.        ]
 ...
 [-73.99220876  40.73867024 -73.99499548 ...   0.           0.
    0.        ]
 [-73.98586354  40.74327496 -73.98936642 ...   0.           0.
    0.        ]
 [-73.98148596  40.7544187  -73.98155982 ...   0.           0.
    0.        ]]


In [10]:
#Standard Scaler benutzen zum vorverarbeiten
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_st = sc.fit_transform(X_train)
X_test_st = sc.transform(x_test)

In [36]:
from sklearn.ensemble import RandomForestRegressor
forr_with_Scaler = RandomForestRegressor(n_estimators=20, random_state=0)
forr_scale = forr_with_Scaler.fit(X_train_st, Y_train)
pred_Forr_Scale = forr_with_Scaler.predict(X_test_st)

In [12]:
print(pred_Forr_Scale)

[[-73.9936719   40.72758905 -73.96143055 ...   0.           0.
    0.        ]
 [-73.9810134   40.7330164  -73.9385932  ...   0.           0.1
    0.        ]
 [-73.97949065  40.75370385 -73.98123725 ...   0.           0.
    0.        ]
 ...
 [-73.994825    40.7365137  -73.99313895 ...   0.           0.
    0.        ]
 [-73.9845751   40.74739865 -73.98511825 ...   0.           0.
    0.        ]
 [-73.9810475   40.7548189  -73.98006755 ...   0.           0.
    0.        ]]


## Evaluating MSE

In [28]:
from sklearn import metrics
#MSE 20
print("MSE (Absolute): ", metrics.mean_absolute_error(y_test, predForr))
print("MSE: ", metrics.mean_squared_error(y_test, predForr))
print("MSE (SQRT): ", np.sqrt(metrics.mean_squared_error(y_test, predForr)))

MSE (Absolute):  0.151493702669237
MSE:  0.1510306424529739
MSE (SQRT):  0.38862661058266956


In [ ]:
print("MSE: ", metrics.mean_squared_error(y_test, Y_train))

In [29]:
#MSE 50
print("MSE (Absolute): ", metrics.mean_absolute_error(y_test, predForr_Default))
print("MSE: ", metrics.mean_squared_error(y_test, predForr_Default))
print("MSE (SQRT): ", np.sqrt(metrics.mean_squared_error(y_test, predForr_Default)))


MSE (Absolute):  0.15012534200015415
MSE:  0.1479230453859393
MSE (SQRT):  0.3846076512316666


In [30]:
#MSE Scaler
print("MSE (Absolute): ", metrics.mean_absolute_error(y_test, pred_Forr_Scale))
print("MSE: ", metrics.mean_squared_error(y_test, pred_Forr_Scale))
print("MSE (SQRT): ", np.sqrt(metrics.mean_squared_error(y_test, pred_Forr_Scale)))

MSE (Absolute):  0.15130648955242773
MSE:  0.15099402738540837
MSE (SQRT):  0.38857949944047276


## Model Error

In [ ]:
#n_estimates 50 regForr_Default
from sklearn.metrics import r2_score

testFehlerForr_Default = forr_with_Scaler.fit(X_train, Y_train).score(X_train, Y_train)
trainingsFehler_Default = forr_with_Scaler.score(x_test, y_test)
print("Testfehler: ",testFehlerForr_Default)
print("Trainingsfehler: ",trainingsFehler_Default)

In [ ]:
from sklearn.metrics import accuracy_score

#Accuracy
acc_50 = accuracy_score(y_test, predForr_Default)
print("Accuracy: ",acc_50, "\n")

In [ ]:
#n_estimates 20
trainingsFehler_20 = regForr.fit(X_train, Y_train).r2_score(X_train, Y_train)
testFehler_20 = regForr.r2_score(x_test, y_test)
print("Testfehler: ",testFehler_20)
print("Trainingsfehler: ",trainingsFehler_20)

In [ ]:
#Accuracy
acc_20 = accuracy_score(y_test, predForr)
print("Accuracy: ",acc_50, "\n")

In [ ]:
#n_estimates 20 und Standardscaler vorab
trainingsFehler_scaler = forr_with_Scaler.fit(X_train, y_train).r2_score(X_train, Y_train)
testFehler_scaler = forr_with_Scaler.r2_score(x_test, y_test)
print("Testfehler: ",testFehler_scaler)
print("Trainingsfehler: ",trainingsFehler_scaler)

In [ ]:
#Accuracy
acc_20 = accuracy_score(y_test, pred_Forr_Scale)
print("Accuracy: ",acc_50, "\n")